# Modelo de Recomendación Item-Item:


El modelo debe establecer una relación ítem-ítem, donde la recomendación de elementos similares se basa en la medida de similitud entre un ítem dado y el resto. En este contexto, el modelo toma como entrada un juego específico y genera como salida una lista de juegos recomendados. Para lograr esto, se recomienda aplicar la métrica de similitud del coseno.

Para lo anterior se hace uso de los datos guardados en la carpeta "Clean_Data" sobre el archivo "SGames_CD.csv" donde desde el inicio se permitió que los datos en genero quedaran en listas sin extender porque podría facilitar el desarrollo del modelo. Aunque no se descarta la opción de extender los generos.

### 1. Importar las librerías necesarias para la creación del modelo y descarga de datos:

In [202]:
import pandas as pd


### 2. Abrir y arreglar los datos para que no hayan registros nulos:

In [203]:
Model_Data = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')
Model_Data.head(2)

,publisher,genres,app_name,release_date,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,['Single-player'],4.99,False,761140.0,Kotoshiro,2018
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,643980.0,Secret Level SRL,2018


In [204]:
Model_Data=Model_Data[['id','genres','publisher','release_date','specs','price','early_access','developer','release_year']]
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   genres        31994 non-null  object 
 2   publisher     24062 non-null  object 
 3   release_date  29781 non-null  object 
 4   specs         31463 non-null  object 
 5   price         30267 non-null  float64
 6   early_access  32133 non-null  bool   
 7   developer     28834 non-null  object 
 8   release_year  32133 non-null  int64  
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 2.0+ MB


Se realizan los cambios para garantizar que no hay datos nulos, se eliminan datos que tengan al mismo tiempo nulos en año y genero, para los demás valores se imputa con desconocido:

In [205]:
# Eliminar registros con valores nulos en 'genres' y valores de 0 en 'release_year'

Model_Data = Model_Data[(Model_Data['release_year'] != 0) & (Model_Data['genres'].notnull())]

# Reemplazar valores nulos en 'genres' con 'desconocido'
Model_Data['genres'] = Model_Data['genres'].fillna('a stranger')

# Reemplazar valores nulos en 'developer' y 'publisher' con 'desconocido':
Model_Data['developer'] = Model_Data['developer'].fillna('a stranger')
Model_Data['publisher'] = Model_Data['publisher'].fillna('a stranger')

# Colocar 0 en los valores nulos de 'price'
Model_Data['price'] = Model_Data['price'].fillna(0)

# Eliminar registros con valores nulos en 'genres'
Model_Data = Model_Data.dropna(subset=['genres'])

# Eliminar la columna 'release_date'
Model_Data = Model_Data.drop(columns=['release_date'])

# Colocar 'not specified' en los valores nulos de la columna 'specs'
Model_Data['specs'] = Model_Data['specs'].fillna('not specified')

# Verificar los cambios
Model_Data.head(-5)


,id,genres,publisher,specs,price,early_access,developer,release_year
0,761140.0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Kotoshiro,['Single-player'],4.99,False,Kotoshiro,2018
1,643980.0,"['Free to Play', 'Indie', 'RPG', 'Strategy']","Making Fun, Inc.","['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Secret Level SRL,2018
2,670290.0,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Poolians.com,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Poolians.com,2017
3,767400.0,"['Action', 'Adventure', 'Casual']",彼岸领域,['Single-player'],0.99,False,彼岸领域,2017
5,772540.0,"['Action', 'Adventure', 'Simulation']",Trickjump Games Ltd,"['Single-player', 'Steam Achievements']",3.99,False,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...
32122,761480.0,"['Adventure', 'Indie']",Phil Fortier,"['Single-player', 'Steam Achievements']",0.99,False,Phil Fortier,2018
32123,771810.0,"['Action', 'Adventure', 'Indie']",Retro Army Limited,"['Single-player', 'Captions available']",0.00,False,Retro Army Limited,2018
32124,767590.0,"['Casual', 'Indie']",OrtiGames/OrtiSoft,"['Single-player', 'Shared/Split Screen', 'Stea...",0.99,False,"Oscar Ortigueira López,OrtiGames/OrtiSoft",2018
32125,747320.0,"['Indie', 'RPG']",INGAME,"['Single-player', 'Steam Achievements', 'Steam...",14.99,False,INGAME,2018


Se obtiene una base sin nulos para trabajar:

In [206]:
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29644 entries, 0 to 32131
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            29644 non-null  float64
 1   genres        29644 non-null  object 
 2   publisher     29644 non-null  object 
 3   specs         29644 non-null  object 
 4   price         29644 non-null  float64
 5   early_access  29644 non-null  bool   
 6   developer     29644 non-null  object 
 7   release_year  29644 non-null  int64  
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 1.8+ MB


In [207]:
# Contar registros con valor 0 en 'release_year'
count_zero_release_year = (Model_Data['release_year'] == 0).sum()

# Imprimir el resultado
print("Cantidad de registros con release_year igual a 0:", count_zero_release_year)


Cantidad de registros con release_year igual a 0: 0


### 3. Desarrollo del modelo "Similitud del Coseno":

3.1. Convertir "genres" y "specs" en cadena para facilitar el desarrollo del modelo:

In [208]:
# Eliminar comillas, corchetes, comas y reemplazar guiones por espacios en 'genres'
Model_Data['genres'] = Model_Data['genres'].astype(str).str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Eliminar comillas, corchetes, comas y reemplazar guiones por espacios en 'specs'
Model_Data['specs'] = Model_Data['specs'].astype(str).str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Verificar los cambios
Model_Data.head()


,id,genres,publisher,specs,price,early_access,developer,release_year
0,761140.0,Action Casual Indie Simulation Strategy,Kotoshiro,Single player,4.99,False,Kotoshiro,2018
1,643980.0,Free to Play Indie RPG Strategy,"Making Fun, Inc.",Single player Multi player Online Multi Player...,0.00,False,Secret Level SRL,2018
2,670290.0,Casual Free to Play Indie Simulation Sports,Poolians.com,Single player Multi player Online Multi Player...,0.00,False,Poolians.com,2017
3,767400.0,Action Adventure Casual,彼岸领域,Single player,0.99,False,彼岸领域,2017
5,772540.0,Action Adventure Simulation,Trickjump Games Ltd,Single player Steam Achievements,3.99,False,Trickjump Games Ltd,2018


3.2. Tokenizar genres y specs para facilitar al modelo el análisis (no se exige en estos modelos pero parece apropiado) también se normaliza. Para developer y Publisher se decide codificar:

In [209]:
# Tokenizar y normalizar 'genres'
Model_Data['genres'] = Model_Data['genres'].astype(str).str.lower().str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Tokenizar y normalizar 'specs'
Model_Data['specs'] = Model_Data['specs'].astype(str).str.lower().str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")


In [210]:
# Codificación de dummies para 'publisher' y 'developer'
#Model_Data = pd.get_dummies(Model_Data, columns=['publisher', 'developer'], drop_first=True)


# Verificar los cambios
Model_Data.head()


,id,genres,publisher,specs,price,early_access,developer,release_year
0,761140.0,action casual indie simulation strategy,Kotoshiro,single player,4.99,False,Kotoshiro,2018
1,643980.0,free to play indie rpg strategy,"Making Fun, Inc.",single player multi player online multi player...,0.00,False,Secret Level SRL,2018
2,670290.0,casual free to play indie simulation sports,Poolians.com,single player multi player online multi player...,0.00,False,Poolians.com,2017
3,767400.0,action adventure casual,彼岸领域,single player,0.99,False,彼岸领域,2017
5,772540.0,action adventure simulation,Trickjump Games Ltd,single player steam achievements,3.99,False,Trickjump Games Ltd,2018


In [211]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combina las columnas 'genres' y 'specs' en una sola columna de texto
Model_Data['combined_features'] = Model_Data['genres'] + ' ' + Model_Data['specs']

# Selecciona las características relevantes (One-Hot Encoded + combined_features)
text_features = pd.get_dummies(Model_Data['combined_features'])
encoded_features = pd.get_dummies(Model_Data[['publisher', 'developer']], prefix=['publisher', 'developer'])
features = pd.concat([encoded_features, text_features], axis=1)

# Utiliza CountVectorizer para convertir la columna 'combined_features' en una matriz de recuentos
vectorizer = CountVectorizer()
text_features_matrix = vectorizer.fit_transform(Model_Data['combined_features'])

# Combina la matriz de recuentos con las características relevantes
all_features_matrix = pd.concat([pd.DataFrame(text_features_matrix.toarray()), features.reset_index(drop=True)], axis=1)

# Calcula la similitud de coseno entre los vectores
cosine_sim = cosine_similarity(all_features_matrix, all_features_matrix)

# Imprime la matriz de similitud de coseno
print(cosine_sim)


[[1.         0.31192515 0.41105415 ... 0.34503278 0.32659863 0.35082321]
 [0.31192515 1.         0.67161876 ... 0.4663724  0.50937163 0.45596075]
 [0.41105415 0.67161876 1.         ... 0.28365431 0.28767798 0.30901572]
 ...
 [0.34503278 0.4663724  0.28365431 ... 1.         0.56343617 0.72627304]
 [0.32659863 0.50937163 0.28767798 ... 0.56343617 1.         0.5728919 ]
 [0.35082321 0.45596075 0.30901572 ... 0.72627304 0.5728919  1.        ]]


In [212]:
import numpy as np

# Elegir un ítem específico (cambiar 'selected_item_id' por el ID deseado)
selected_item_id = 	670290

# Obtener la fila correspondiente al ítem seleccionado
selected_item_index = Model_Data[Model_Data['id'] == selected_item_id].index[0]

# Calcular la similitud de coseno entre el ítem seleccionado y todos los demás ítems
cosine_similarities = cosine_sim[selected_item_index]

# Obtener los índices de los ítems más similares (excluyendo el propio ítem seleccionado)
similar_items_indices = np.argsort(cosine_similarities)[::-1][1:6]

# Obtener los IDs de los ítems más similares
similar_item_ids = Model_Data.loc[similar_items_indices, 'id'].tolist()

# Imprimir los IDs de los ítems más similares
print("Top 5 ítems más similares:")
print(similar_item_ids)


Top 5 ítems más similares:
[614300.0, 605490.0, 608590.0, 505750.0, 492957.0]
